In [1]:
import tensorflow as tf
import cv2
from keras.layers import Dense
from tensorflow import keras
# import tensorflow_federated as tff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imutils import paths

In [2]:
from typing import Union

import tqdm
import os
import sys
import random
import pathlib

In [3]:
IMG_SIZE = 256
BATCH_SIZE = 64
EPOCHS = 200

MAX_SEQ_LENGTH = 20  # might not use
NUM_FEATUIRES = 2048

In [4]:
def load_video(path, max_frames=None, resize=None):
    npy_path = pathlib.Path(path).with_suffix('.npy')
    if os.path.isfile(npy_path):
      if resize is None:
        print("Falling back to resize=(IMG_SIZE, IMG_SIZE)")
        resize = (IMG_SIZE, IMG_SIZE)
      frames = np.load(npy_path, allow_pickle=True)
      return np.array([cv2.resize(frame, resize) for frame in frames])

    cap =  cv2.VideoCapture(path)
    try:
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if max_frames and len(frames) >= max_frames:
                break
            if resize is not None:
              frame = cv2.resize(frame, resize)
            frames.append(frame)
    finally:
        cap.release()
    return np.array(frames)

In [5]:
def video_frame_generator(path, resize=(IMG_SIZE, IMG_SIZE)):

    cap =  cv2.VideoCapture(path)
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            yield frame
    finally:
        cap.release()

In [6]:
def build_feature_extractor(model='InceptionV3'):
    extractor_dict = \
        {
            'InceptionV3': keras.applications.inception_v3.InceptionV3,
            'VGG16': keras.applications.vgg16.VGG16,
            'VGG19': keras.applications.vgg19.VGG19,
            'ResNet50': keras.applications.resnet.ResNet50,
            'Xception': keras.applications.xception.Xception,
            'InceptionResNetV2': keras.applications.inception_resnet_v2.InceptionResNetV2,
            'MobileNet': keras.applications.mobilenet.MobileNet,
            'MobileNetV2': keras.applications.mobilenet_v2.MobileNetV2,
            'DenseNet121': keras.applications.densenet.DenseNet121,
            'DenseNet169': keras.applications.densenet.DenseNet169,
            'DenseNet201': keras.applications.densenet.DenseNet201,
            'EfficientNetB0': keras.applications.efficientnet.EfficientNetB0,
            'EfficientNetB1': keras.applications.efficientnet.EfficientNetB1,
            'EfficientNetB2': keras.applications.efficientnet.EfficientNetB2,
            'EfficientNetB3': keras.applications.efficientnet.EfficientNetB3,
            'EfficientNetB4': keras.applications.efficientnet.EfficientNetB4,
            'EfficientNetB5': keras.applications.efficientnet.EfficientNetB5,
            'EfficientNetB6': keras.applications.efficientnet.EfficientNetB6,
            'EfficientNetB7': keras.applications.efficientnet.EfficientNetB7,
        }

    try:
        feature_extractor = extractor_dict[model](include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg')
    except KeyError:
        print("Model not found")
        print("Falling back to InceptionV3")
        feature_extractor = extractor_dict['InceptionV3'](include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg')
    preprocess_dict = \
        {
            'InceptionV3': keras.applications.inception_v3.preprocess_input,
            'VGG16': keras.applications.vgg16.preprocess_input,
            'VGG19': keras.applications.vgg19.preprocess_input,
            'ResNet50': keras.applications.resnet.preprocess_input,
            'Xception': keras.applications.xception.preprocess_input,
            'InceptionResNetV2': keras.applications.inception_resnet_v2.preprocess_input,
            'MobileNet': keras.applications.mobilenet.preprocess_input,
            'MobileNetV2': keras.applications.mobilenet_v2.preprocess_input,
            'DenseNet121': keras.applications.densenet.preprocess_input,
            'DenseNet169': keras.applications.densenet.preprocess_input,
            'DenseNet201': keras.applications.densenet.preprocess_input,
            'NASNetLarge': keras.applications.nasnet.preprocess_input,
            'NASNetMobile': keras.applications.nasnet.preprocess_input,
            'EfficientNetB0': keras.applications.efficientnet.preprocess_input,
            'EfficientNetB1': keras.applications.efficientnet.preprocess_input,
            'EfficientNetB2': keras.applications.efficientnet.preprocess_input,
            'EfficientNetB3': keras.applications.efficientnet.preprocess_input,
            'EfficientNetB4': keras.applications.efficientnet.preprocess_input,
            'EfficientNetB5': keras.applications.efficientnet.preprocess_input,
            'EfficientNetB6': keras.applications.efficientnet.preprocess_input,
            'EfficientNetB7': keras.applications.efficientnet.preprocess_input,
        }
    try:
        preprocess = preprocess_dict[model]
    except KeyError:
        print("Model not found")
        print("Falling back to InceptionV3")
        preprocess = preprocess_dict['InceptionV3']

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess(inputs)
    features = feature_extractor(preprocessed)

    return tf.keras.Model(inputs, features)

In [7]:
def build_autoencoder(feature_extractor: tf.keras.Model = None, units=None, units_mask=None, initial_n_frames: int = MAX_SEQ_LENGTH) -> tf.keras.Model:
    if units is None or units == []:
       units = [128]

    # Inputs
    inputs = tf.keras.layers.Input(shape=(initial_n_frames, feature_extractor.output_shape[1]))

    if len(units) == 1:
        # Only one encoder and decoder
        encoder = tf.keras.layers.LSTM(units=units[0], return_state=True)
        encoder_outputs, state_h, state_c = encoder(inputs)

        decoder_lstm = tf.keras.layers.LSTM(units=units[0], return_state=True, return_sequences=False)
        decoder_outputs, _, _ = decoder_lstm(inputs, initial_state=[state_h, state_c])
    else:
        internal_states = []
        # Encoder

        # First encoder
        encoder = tf.keras.layers.LSTM(units=units[0], return_state=True, return_sequences=True, name='encoder_0')
        encoder_outputs, state_h, state_c = encoder(inputs)
        internal_states.append([state_h, state_c])

        # Residual encoder and decoder
        for i, unit in enumerate(units[1:-1]):
           encoder = tf.keras.layers.LSTM(units=unit, return_state=True, return_sequences=True, name=f'encoder_{i+1}')
           encoder_outputs, state_h, state_c = encoder(encoder_outputs)
           internal_states.append([state_h, state_c])

        # Last encoder
        encoder = tf.keras.layers.LSTM(units=units[-1], return_state=True, return_sequences=False, name=f'encoder_{len(units)-1}')
        encoder_outputs, state_h, state_c = encoder(encoder_outputs)
        internal_states.append([state_h, state_c])

        # Decoder

        # First decoder
        decoder = tf.keras.layers.LSTM(units=units[-1], return_state=True, return_sequences=True, name='decoder_0')
        if units_mask[0]:
            decoder_outputs, _, _ = decoder(inputs, initial_state=internal_states[-1])
        else:
            decoder_outputs, _, _ = decoder(inputs)

        for i, (unit, internal_state, mask) in enumerate(zip(units[0:-1:-1], internal_states[0:-1:-1], units_mask)):
            decoder = tf.keras.layers.LSTM(units=unit, return_sequences=True, name=f'decoder_{i+1}')
            if mask:
                decoder_outputs = decoder(decoder_outputs, initial_state=internal_state)
            else:
                decoder_outputs = decoder(decoder_outputs)

        # Last decoder
        decoder = tf.keras.layers.LSTM(units=units[0], return_state=True, return_sequences=False, name=f'decoder_{len(units)-1}')
        if units_mask[-1]:
            decoder_outputs, _, _ = decoder(decoder_outputs, initial_state=internal_states[0])
        else:
            decoder_outputs, _, _ = decoder(decoder_outputs)

    decoder_dense = Dense(units=feature_extractor.output_shape[1], activation='relu')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = tf.keras.Model(inputs=inputs, outputs=decoder_outputs)
    return model

In [16]:
class ULSTMModel(tf.keras.Model):

    def __init__(self, feature_extractor: str,
                 autoencoder_units,
                 initial_n_frames: int = MAX_SEQ_LENGTH,
                 units_mask=None,
                 finetune_feature_extractor: bool = False,
                 **kwargs):
        super(ULSTMModel, self).__init__(**kwargs)
        self.feature_extractor = build_feature_extractor(feature_extractor)
        self.feature_extractor.trainable = False
        self.autoencoder = build_autoencoder(feature_extractor=self.feature_extractor, units=autoencoder_units, units_mask=units_mask, initial_n_frames=initial_n_frames)

    def __call__(self, inputs, training=True):
        outputs = self.autoencoder(inputs, training=training)
        return outputs


In [9]:
colab_used = False
try:
  from google.colab import drive
  colab_used = True
except ImportError:
  pass

if colab_used:
  drive.mount('/gdrive', force_remount=True)
  UCF_Crime_NormalVideos_path = '/gdrive/MyDrive/Colab Notebooks/Ongoing Work/dataset/Training-Normal-Videos-Part-1'
else:
  UCF_Crime_NormalVideos_path = '../../dataset/UCF-Crime/Training-Normal-Videos-Part-1'

UCF_Crime_TrainingNormalVideos = os.listdir(UCF_Crime_NormalVideos_path)
UCF_Crime_TrainingNormalVideos = [os.path.join(UCF_Crime_NormalVideos_path, video) for video in UCF_Crime_TrainingNormalVideos]
random.shuffle(UCF_Crime_TrainingNormalVideos)

# 20% of the videos will be used for validation
UCF_Crime_ValidationNormalVideos = UCF_Crime_TrainingNormalVideos[:int(len(UCF_Crime_TrainingNormalVideos) * 0.2)]

# 80% of the dataset for training
UCF_Crime_TrainingNormalVideos = UCF_Crime_TrainingNormalVideos[int(len(UCF_Crime_TrainingNormalVideos) * 0.2):]

npz_dataset_path = '/gdrive/MyDrive/Colab Notebooks/Ongoing Work/dataset/' if colab_used else '../../dataset/'

Mounted at /gdrive


In [10]:
"""for video in tqdm.tqdm(UCF_Crime_ValidationNormalVideos + UCF_Crime_TrainingNormalVideos):
  path = pathlib.Path(video)
  try:
    np.save(path.with_suffix('.npy'), load_video(video, max_frames=3000)) 
  except OSError:
    if colab_used:
      drive.mount('/gdrive', force_remount=True)"""

"for video in tqdm.tqdm(UCF_Crime_ValidationNormalVideos + UCF_Crime_TrainingNormalVideos):\n  path = pathlib.Path(video)\n  try:\n    np.save(path.with_suffix('.npy'), load_video(video, max_frames=3000)) \n  except OSError:\n    if colab_used:\n      drive.mount('/gdrive', force_remount=True)"

In [12]:
class DataGenerator(tf.keras.utils.Sequence):

    def __init__(self, videos, batch_size, n_frames, shuffle=True, resize=(IMG_SIZE, IMG_SIZE), feature_extractor=None, samples_per_video=2, partition='training'):
        self.samples_per_video = samples_per_video
        self.batch_size = batch_size
        self.n_frames = n_frames
        self.shuffle = shuffle
        self.resize = resize
        if feature_extractor is None:
          raise NotImplementedError("Pass a feature extractor")
        else:
          self.feature_extractor = build_feature_extractor(feature_extractor)
        self.partition = partition
        self.__initVideoData(videos, feature_extractor)
        self.data_length = len(videos) * self.samples_per_video
        self.on_epoch_end()


    def __initVideoData(self, videos, feature_extractor):
      X_path = os.path.join(npz_dataset_path, f'X_{feature_extractor}_{self.resize}{self.n_frames}_{self.partition}.npy')
      y_path = os.path.join(npz_dataset_path, f'y_{feature_extractor}_{self.resize}{self.n_frames}_{self.partition}.npy')
      print(f'{X_path} exists: {os.path.isfile(X_path)}')
      print(f'{y_path} exists: {os.path.isfile(y_path)}')
      if os.path.isfile(X_path) and os.path.isfile(y_path):
        self.X = np.load(X_path)
        self.y = np.load(y_path)
      else:
        self.X = np.empty((len(videos) * self.samples_per_video, self.n_frames, self.feature_extractor.output_shape[-1]))
        self.y = np.empty((len(videos) * self.samples_per_video, self.feature_extractor.output_shape[-1]))
        for v, video in enumerate(tqdm.tqdm(videos)):
          frames = load_video(video, resize=self.resize, max_frames=self.samples_per_video*self.n_frames)
          if len(frames) <= self.n_frames:
            continue
          # Take sample from the current video
          for s in range(self.samples_per_video):
            start_frame = np.random.randint(0, len(frames) - self.n_frames - 1)
            end_frame = start_frame + self.n_frames
            for j, frame in enumerate(frames[start_frame:end_frame]):
              self.X[v * self.samples_per_video + s, j, :] = self.feature_extractor(frame[np.newaxis])
            self.y[v * self.samples_per_video + s, :] = self.feature_extractor(frames[end_frame][np.newaxis])
        np.save(os.path.join(npz_dataset_path, f'X_{feature_extractor}_{self.resize}{self.n_frames}_{self.partition}'), self.X)
        np.save(os.path.join(npz_dataset_path, f'y_{feature_extractor}_{self.resize}{self.n_frames}_{self.partition}'), self.y)

    def __len__(self):
        return int(np.ceil(self.data_length / self.batch_size))

    def __getitem__(self, index):
        X = self.X[index * self.batch_size: (index + 1) * self.batch_size, :, :]
        y = self.y[index * self.batch_size: (index + 1) * self.batch_size, :]
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(self.data_length)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

feature_extractor='EfficientNetB0'
initial_n_frames=32
train_data_generator_params = {'videos': UCF_Crime_TrainingNormalVideos, 'batch_size': 32, 'n_frames': initial_n_frames, 'shuffle': True, 'feature_extractor': feature_extractor}
validation_data_generator_params = {'videos': UCF_Crime_ValidationNormalVideos, 'batch_size': 32, 'n_frames': initial_n_frames, 'shuffle': False, 'feature_extractor': feature_extractor, 'partition': 'validation'}

training_generator = DataGenerator(**train_data_generator_params)
validation_generator = DataGenerator(**validation_data_generator_params)

print(len(training_generator))


/gdrive/MyDrive/Colab Notebooks/Ongoing Work/dataset/X_EfficientNetB0_(256, 256)32_training.npy exists: False
/gdrive/MyDrive/Colab Notebooks/Ongoing Work/dataset/y_EfficientNetB0_(256, 256)32_training.npy exists: False


100%|██████████| 688/688 [2:54:23<00:00, 15.21s/it]


/gdrive/MyDrive/Colab Notebooks/Ongoing Work/dataset/X_EfficientNetB0_(256, 256)32_validation.npy exists: False
/gdrive/MyDrive/Colab Notebooks/Ongoing Work/dataset/y_EfficientNetB0_(256, 256)32_validation.npy exists: False


100%|██████████| 171/171 [40:26<00:00, 14.19s/it]


43


In [13]:
def lr_scheduler(epoch, lr):
    lr_list = [1e-2] * 40 + [1e-3] * 50 + [1e-4] * 100 + [1e-5] * 10

    return lr_list[epoch]

In [20]:
models_path = '/gdrive/MyDrive/Colab Notebooks/Ongoing Work/models' if colab_used else '../../models/UCF-Crime'
autoencoder_units=[128, 64, 32]
units_mask=[True, True, True]
model_instance = ULSTMModel(feature_extractor=feature_extractor,
                             autoencoder_units=autoencoder_units,
                             initial_n_frames=initial_n_frames,
                             units_mask=units_mask,
                             finetune_feature_extractor=False, 
                             name=f'ULSTM_{feature_extractor}_{initial_n_frames}')

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath=f'{models_path}/{feature_extractor}/ULSTM{initial_n_frames}_{"-".join(str(x) for x in zip(autoencoder_units, units_mask))}',
                                    monitor='val_loss',
                                    save_best_only=True, save_weights_only=True, save_format='tf'),
    keras.callbacks.TensorBoard(log_dir=f"{models_path}/logs/{feature_extractor}/ULSTM{initial_n_frames}", write_graph=True, write_images=True),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='auto'),
    keras.callbacks.TerminateOnNaN(),
    keras.callbacks.LearningRateScheduler(lr_scheduler),
    keras.callbacks.BackupAndRestore(backup_dir=f'{models_path}/BackupAndRestore/{feature_extractor}/ULSTM{initial_n_frames}_{"-".join(str(x) for x in zip(autoencoder_units, units_mask))}'),
]

In [21]:
model_instance.compile(optimizer='adam', loss='mse', 
                       metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.CosineSimilarity(), tf.keras.metrics.LogCoshError()],
                       run_eagerly=True)
model_instance.fit(x=training_generator,
                   validation_data=validation_generator,
                   epochs=EPOCHS,
                   callbacks=callbacks)

Epoch 1/200
43/43 [==============================] - 5s 121ms/step - loss: 0.0866 - mean_absolute_error: 0.2007 - cosine_similarity: 0.5765 - logcosh: 0.0394 - val_loss: nan - val_mean_absolute_error: nan - val_cosine_similarity: nan - val_logcosh: nan - lr: 0.0100
Epoch 2/200
43/43 [==============================] - 5s 119ms/step - loss: 0.0729 - mean_absolute_error: 0.1870 - cosine_similarity: 0.6640 - logcosh: 0.0337 - val_loss: nan - val_mean_absolute_error: nan - val_cosine_similarity: nan - val_logcosh: nan - lr: 0.0100
Epoch 3/200
43/43 [==============================] - 5s 124ms/step - loss: 0.0663 - mean_absolute_error: 0.1794 - cosine_similarity: 0.7002 - logcosh: 0.0308 - val_loss: nan - val_mean_absolute_error: nan - val_cosine_similarity: nan - val_logcosh: nan - lr: 0.0100
Epoch 4/200
43/43 [==============================] - 5s 125ms/step - loss: 0.0624 - mean_absolute_error: 0.1752 - cosine_similarity: 0.7204 - logcosh: 0.0292 - val_loss: nan - val_mean_absolute_error: n

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/Xception/ULSTM16